# 01 — Logic Families and Voltage Levels

**Module 06: Integrated Circuits**

---

In the previous notebook we built logic gates from individual transistors. In practice, you buy logic gates as **integrated circuits** (ICs) that come in standardized **logic families** with guaranteed voltage levels, speed, and power characteristics.

This notebook covers the practical world of logic ICs. By the end, you will:
- Understand what a logic family is and why standardization matters
- Know the voltage thresholds for TTL and CMOS families
- Calculate noise margins and understand why they matter
- Know when and why to use level shifting between 3.3V and 5V logic
- Understand fan-out, propagation delay, and decoupling capacitors
- Wire up a real 74HC00 NAND gate IC and verify its truth table

## Concept — What Is a Logic Family?

A **logic family** is a set of ICs designed to work together, sharing:
- **Supply voltage** (e.g., 5V, 3.3V)
- **Input/output voltage thresholds** (what counts as HIGH or LOW)
- **Output drive capability** (how much current an output can source/sink)
- **Speed characteristics** (propagation delay, rise/fall times)
- **Power consumption** profile

### Major logic families

| Family | Technology | Supply | Speed | Power | Era |
|--------|-----------|--------|-------|-------|-----|
| 74 (standard TTL) | BJT | 5V | 10 ns | 10 mW/gate | 1960s |
| 74LS (Low-power Schottky) | BJT | 5V | 9.5 ns | 2 mW/gate | 1970s |
| 74HC (High-speed CMOS) | CMOS | 2-6V | 8 ns | ~0.5 uW static | 1980s |
| 74HCT (HC with TTL inputs) | CMOS | 5V | 12 ns | ~0.5 uW static | 1980s |
| 74AHC (Advanced HC) | CMOS | 2-5.5V | 3.7 ns | ~0.5 uW static | 1990s |
| 74LVC (Low-voltage CMOS) | CMOS | 1.65-3.6V | 3.3 ns | ~0.5 uW static | 2000s |

The number after "74" indicates the function (00 = quad NAND, 04 = hex inverter, etc.). The letters in the middle indicate the logic family.

```
  74  HC  00
  ^^  ^^  ^^
  |   |   +-- Function: Quad 2-input NAND
  |   +------ Family: High-speed CMOS
  +---------- Series: 7400 (commercial temp range)
```

## Concept — TTL Voltage Levels

**TTL (Transistor-Transistor Logic)** was the original 7400 series, built from bipolar junction transistors (BJTs). It defined the voltage levels that became the industry standard for decades:

```
5.0V _______________
                     Output HIGH: V_OH >= 2.4V
2.4V - - - - - - - -
                     "Forbidden zone" (undefined)
2.0V - - - - - - - -
                     Input HIGH threshold: V_IH >= 2.0V
     ...
0.8V - - - - - - - -
                     Input LOW threshold: V_IL <= 0.8V
0.4V - - - - - - - -
                     Output LOW: V_OL <= 0.4V
0.0V _______________
```

### Key TTL specs:
| Parameter | Min/Max | Meaning |
|-----------|---------|--------|
| V_OH | >= 2.4V | Output HIGH is at least 2.4V |
| V_OL | <= 0.4V | Output LOW is at most 0.4V |
| V_IH | >= 2.0V | Input recognizes >= 2.0V as HIGH |
| V_IL | <= 0.8V | Input recognizes <= 0.8V as LOW |

### Noise margins:
- **NM_H** = V_OH - V_IH = 2.4V - 2.0V = **0.4V** (noise room for HIGH signals)
- **NM_L** = V_IL - V_OL = 0.8V - 0.4V = **0.4V** (noise room for LOW signals)

A noise margin of 0.4V means the signal can be corrupted by up to 0.4V of noise and still be read correctly. This is not a lot!

## Concept — CMOS Voltage Levels (74HC Series)

CMOS logic (74HC family) has **much better** voltage levels:

```
VCC  _______________
                     Output HIGH: V_OH >= VCC - 0.1V (nearly VCC!)
VCC-0.1V - - - - - -
                     
0.7*VCC - - - - - - -
                     Input HIGH threshold: V_IH >= 0.7 * VCC
     ...
0.3*VCC - - - - - - -
                     Input LOW threshold: V_IL <= 0.3 * VCC
0.1V - - - - - - - -
                     Output LOW: V_OL <= 0.1V (nearly 0V!)
0.0V _______________
```

### Key CMOS (74HC) specs at VCC = 5V:
| Parameter | Value | Meaning |
|-----------|-------|--------|
| V_OH | >= 4.9V | Output HIGH is nearly VCC (rail-to-rail!) |
| V_OL | <= 0.1V | Output LOW is nearly GND |
| V_IH | >= 3.5V | Input recognizes >= 3.5V as HIGH |
| V_IL | <= 1.5V | Input recognizes <= 1.5V as LOW |

### Noise margins at 5V:
- **NM_H** = V_OH - V_IH = 4.9V - 3.5V = **1.4V**
- **NM_L** = V_IL - V_OL = 1.5V - 0.1V = **1.4V**

That is 3.5x better noise margin than TTL! CMOS outputs swing nearly rail-to-rail, leaving maximum room for noise.

### The catch: CMOS thresholds scale with VCC

At VCC = 3.3V:
- V_IH >= 0.7 * 3.3V = 2.31V
- V_IL <= 0.3 * 3.3V = 0.99V

This means a 3.3V CMOS output (HIGH = 3.3V) cannot reliably drive a 5V CMOS input (needs >= 3.5V). This is the **level-shifting problem**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Visualize TTL vs CMOS voltage levels with noise margins

fig, axes = plt.subplots(1, 3, figsize=(16, 7))

families = [
    {
        'name': 'TTL (74LS)\nVCC = 5V',
        'vcc': 5.0,
        'VOH': 2.7, 'VOH_min': 2.4,
        'VOL': 0.35, 'VOL_max': 0.5,
        'VIH': 2.0,
        'VIL': 0.8,
    },
    {
        'name': 'CMOS (74HC)\nVCC = 5V',
        'vcc': 5.0,
        'VOH': 4.9, 'VOH_min': 4.9,
        'VOL': 0.1, 'VOL_max': 0.1,
        'VIH': 3.5,
        'VIL': 1.5,
    },
    {
        'name': 'CMOS (74HC)\nVCC = 3.3V',
        'vcc': 3.3,
        'VOH': 3.2, 'VOH_min': 3.2,
        'VOL': 0.1, 'VOL_max': 0.1,
        'VIH': 2.31,
        'VIL': 0.99,
    },
]

for ax, fam in zip(axes, families):
    vcc = fam['vcc']
    
    # Draw voltage rail bars using Rectangle patches for precise positioning
    # Output LOW region (0 to VOL_max)
    ax.barh(0, 1, height=fam['VOL_max'], left=0, color='#EF5350', alpha=0.7, label='Output LOW')
    # Output HIGH region (VOH_min to VCC)
    ax.barh(fam['VOH_min'], 1, height=vcc - fam['VOH_min'], left=0, color='#66BB6A', alpha=0.7, label='Output HIGH')
    
    # Forbidden zone (VIL to VIH)
    ax.barh(fam['VIL'], 1, height=fam['VIH'] - fam['VIL'], left=0,
            color='#FFC107', alpha=0.4, label='Undefined zone')
    
    # Noise margin bands
    NMH = fam['VOH_min'] - fam['VIH']
    NML = fam['VIL'] - fam['VOL_max']
    
    ax.barh(fam['VIH'], 0.3, height=NMH, left=1.1,
            color='#42A5F5', alpha=0.6)
    ax.text(1.25, fam['VIH'] + NMH/2, f'NM_H\n{NMH:.1f}V', ha='center', va='center',
            fontsize=9, fontweight='bold', color='#1565C0')
    
    ax.barh(fam['VOL_max'], 0.3, height=NML, left=1.1,
            color='#42A5F5', alpha=0.6)
    ax.text(1.25, fam['VOL_max'] + NML/2, f'NM_L\n{NML:.1f}V', ha='center', va='center',
            fontsize=9, fontweight='bold', color='#1565C0')
    
    # Threshold labels
    ax.axhline(y=fam['VOH_min'], color='green', linestyle='--', alpha=0.7, xmin=0, xmax=0.6)
    ax.text(-0.05, fam['VOH_min'], f'V_OH={fam["VOH_min"]:.1f}V', ha='right', va='center', fontsize=9, color='green')
    
    ax.axhline(y=fam['VIH'], color='orange', linestyle='--', alpha=0.7, xmin=0, xmax=0.6)
    ax.text(-0.05, fam['VIH'], f'V_IH={fam["VIH"]:.1f}V', ha='right', va='center', fontsize=9, color='orange')
    
    ax.axhline(y=fam['VIL'], color='orange', linestyle='--', alpha=0.7, xmin=0, xmax=0.6)
    ax.text(-0.05, fam['VIL'], f'V_IL={fam["VIL"]:.1f}V', ha='right', va='center', fontsize=9, color='orange')
    
    ax.axhline(y=fam['VOL_max'], color='red', linestyle='--', alpha=0.7, xmin=0, xmax=0.6)
    ax.text(-0.05, fam['VOL_max'], f'V_OL={fam["VOL_max"]:.1f}V', ha='right', va='center', fontsize=9, color='red')
    
    ax.axhline(y=vcc, color='black', linestyle='-', alpha=0.3)
    ax.text(0.5, vcc + 0.15, f'VCC = {vcc}V', ha='center', fontsize=10, fontweight='bold')
    
    ax.set_xlim(-0.8, 1.6)
    ax.set_ylim(-0.3, vcc + 0.5)
    ax.set_title(fam['name'], fontsize=12, fontweight='bold')
    ax.set_ylabel('Voltage (V)' if ax == axes[0] else '', fontsize=11)
    ax.set_xticks([])

fig.suptitle('Logic Family Voltage Levels and Noise Margins', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print('Key observations:')
print('  TTL noise margins are only 0.4V -- tight!')
print('  CMOS at 5V has 1.4V noise margins -- 3.5x better.')
print('  CMOS at 3.3V has smaller absolute margins but still proportionally good.')
print()
print('PROBLEM: 3.3V CMOS output HIGH (3.2V) is BELOW 5V CMOS V_IH (3.5V)!')
print('  --> You cannot directly connect 3.3V CMOS output to 5V CMOS input.')
print('  --> This is why level shifters exist.')

## Concept — 74HCT: The TTL-Compatible CMOS

The **74HCT** family was created to solve a specific problem: replacing TTL chips with CMOS chips without redesigning the board.

74HCT uses:
- CMOS technology (low power, rail-to-rail outputs)
- **TTL-compatible input thresholds** (V_IH = 2.0V, V_IL = 0.8V)

This means:
- A TTL output (V_OH >= 2.4V) can reliably drive a 74HCT input (V_IH = 2.0V)
- A 3.3V CMOS output (V_OH = 3.2V) can also drive a 74HCT input (3.2V > 2.0V)

### The level-shifting cheat sheet

| From | To | Works directly? | Solution |
|------|----|----------------|----------|
| 5V TTL | 5V CMOS (HC) | NO (V_OH=2.4V < V_IH=3.5V) | Use 74HCT or pull-up |
| 5V CMOS | 5V TTL | YES (V_OH=4.9V > V_IH=2.0V) | Direct connection |
| 5V CMOS | 5V CMOS | YES | Direct connection |
| 3.3V CMOS | 5V CMOS | NO (V_OH=3.2V < V_IH=3.5V) | Use level shifter |
| 5V CMOS | 3.3V CMOS | DANGEROUS (exceeds 3.3V max input) | Use level shifter |
| 3.3V CMOS | 5V HCT | YES (V_OH=3.2V > V_IH=2.0V) | Direct connection |

### Level shifting techniques

1. **Voltage divider** (5V to 3.3V): Simple resistors, but slow and wastes power
2. **Series resistor + clamp diode**: Protection for 3.3V inputs from 5V signals
3. **Dedicated level shifter IC** (e.g., TXB0108): Best for bidirectional signals
4. **74HCT as level shifter**: 3.3V logic drives HCT inputs, HCT outputs at 5V

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Noise margin calculator for different logic families and supply voltages

def calc_noise_margins(family_name, VOH, VOL, VIH, VIL):
    NMH = VOH - VIH
    NML = VIL - VOL
    return NMH, NML

# Define families
families = {
    'Standard TTL\n(74)': {'VOH': 2.4, 'VOL': 0.4, 'VIH': 2.0, 'VIL': 0.8},
    'Low-power Schottky\n(74LS)': {'VOH': 2.7, 'VOL': 0.5, 'VIH': 2.0, 'VIL': 0.8},
    'CMOS 5V\n(74HC)': {'VOH': 4.9, 'VOL': 0.1, 'VIH': 3.5, 'VIL': 1.5},
    'CMOS/TTL 5V\n(74HCT)': {'VOH': 4.9, 'VOL': 0.1, 'VIH': 2.0, 'VIL': 0.8},
    'CMOS 3.3V\n(74HC)': {'VOH': 3.2, 'VOL': 0.1, 'VIH': 2.31, 'VIL': 0.99},
}

names = list(families.keys())
NMH_vals = []
NML_vals = []

print(f'{"Family":<25} {"NM_H (V)":>10} {"NM_L (V)":>10} {"Min NM":>10}')
print('=' * 60)
for name, specs in families.items():
    nmh, nml = calc_noise_margins(name, **specs)
    NMH_vals.append(nmh)
    NML_vals.append(nml)
    clean_name = name.replace('\n', ' ')
    print(f'{clean_name:<25} {nmh:>10.2f} {nml:>10.2f} {min(nmh,nml):>10.2f}')

fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(names))
width = 0.35

bars1 = ax.bar(x - width/2, NMH_vals, width, label='NM_H (HIGH noise margin)',
               color='#66BB6A', edgecolor='black', linewidth=0.8)
bars2 = ax.bar(x + width/2, NML_vals, width, label='NM_L (LOW noise margin)',
               color='#EF5350', edgecolor='black', linewidth=0.8)

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{height:.2f}V', ha='center', va='bottom', fontsize=9, fontweight='bold')
for bar in bars2:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
            f'{height:.2f}V', ha='center', va='bottom', fontsize=9, fontweight='bold')

ax.set_xlabel('Logic Family', fontsize=12)
ax.set_ylabel('Noise Margin (V)', fontsize=12)
ax.set_title('Noise Margins by Logic Family', fontsize=13, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(names, fontsize=9)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# How CMOS thresholds change with supply voltage

VCC_range = np.linspace(2.0, 6.0, 100)

# 74HC thresholds scale with VCC
VIH = 0.7 * VCC_range
VIL = 0.3 * VCC_range
VOH = VCC_range - 0.1  # Nearly rail-to-rail
VOL = np.ones_like(VCC_range) * 0.1

NMH = VOH - VIH
NML = VIL - VOL

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Thresholds vs VCC
ax1.plot(VCC_range, VOH, 'g-', linewidth=2.5, label='V_OH (output HIGH)')
ax1.plot(VCC_range, VIH, 'g--', linewidth=2, label='V_IH (input HIGH threshold)')
ax1.plot(VCC_range, VIL, 'r--', linewidth=2, label='V_IL (input LOW threshold)')
ax1.plot(VCC_range, VOL, 'r-', linewidth=2.5, label='V_OL (output LOW)')

ax1.fill_between(VCC_range, VIH, VOH, alpha=0.1, color='green', label='HIGH noise margin')
ax1.fill_between(VCC_range, VOL, VIL, alpha=0.1, color='red', label='LOW noise margin')

# Mark common supply voltages
for vcc, label in [(3.3, '3.3V'), (5.0, '5V')]:
    ax1.axvline(x=vcc, color='gray', linestyle=':', alpha=0.5)
    ax1.text(vcc + 0.05, 0.2, label, fontsize=9, color='gray', rotation=90)

ax1.set_xlabel('Supply Voltage VCC (V)', fontsize=12)
ax1.set_ylabel('Voltage (V)', fontsize=12)
ax1.set_title('74HC CMOS Thresholds vs Supply Voltage', fontsize=13, fontweight='bold')
ax1.legend(fontsize=9, loc='upper left')
ax1.grid(True, alpha=0.3)

# Noise margins vs VCC
ax2.plot(VCC_range, NMH, 'g-', linewidth=2.5, label='NM_H')
ax2.plot(VCC_range, NML, 'r-', linewidth=2.5, label='NM_L')

# TTL noise margins for reference
ax2.axhline(y=0.4, color='blue', linestyle='--', alpha=0.5, label='TTL noise margin (0.4V)')

ax2.set_xlabel('Supply Voltage VCC (V)', fontsize=12)
ax2.set_ylabel('Noise Margin (V)', fontsize=12)
ax2.set_title('74HC Noise Margins vs Supply Voltage', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

for vcc in [3.3, 5.0]:
    ax2.axvline(x=vcc, color='gray', linestyle=':', alpha=0.5)

plt.tight_layout()
plt.show()

print('CMOS noise margins scale linearly with VCC.')
print('Even at 2V, CMOS noise margins exceed TTL noise margins.')
print('This is a fundamental advantage of CMOS: rail-to-rail outputs.')

## The Material Science Why — TTL vs CMOS Transistors

The difference between TTL and CMOS logic families comes directly from the underlying transistor physics:

### TTL: BJT-Based

- Uses bipolar junction transistors (NPN and PNP)
- BJTs are **current-controlled**: base current sets collector current
- A conducting BJT has a **saturation voltage** of ~0.2-0.4V (V_CE(sat))
- This is why V_OL for TTL is 0.4V, not 0V
- The output HIGH uses a "totem-pole" arrangement with a pull-up transistor
- V_OH is limited to ~3.5V because of voltage drops in the pull-up
- **Constant base current is needed** to keep transistors ON --> static power consumption

### CMOS: MOSFET-Based

- Uses complementary NMOS and PMOS MOSFETs
- MOSFETs are **voltage-controlled**: gate voltage sets channel conductivity
- A fully-on MOSFET has very low R_DS(on), so output reaches nearly 0V or VCC
- This is why CMOS outputs are **rail-to-rail**
- The gate is insulated (SiO2) --> **zero static gate current**
- Only charging/discharging gate capacitance consumes power --> switching power only

### The result

| Property | TTL (BJT) | CMOS (MOSFET) |
|----------|-----------|---------------|
| Output swing | 0.4V to ~3.5V | ~0V to ~VCC |
| Static power per gate | ~1-10 mW | ~0.001 mW |
| Input impedance | Low (~kOhms) | Extremely high (>10^12 ohm) |
| Fan-out | ~10 (limited by input current) | ~50+ (limited by capacitance) |
| Speed (1980s) | Faster | Slower |
| Speed (today) | Obsolete | Dominant |

CMOS won because its zero static power allows billions of transistors on one chip. TTL would require kilowatts just to idle.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Power consumption: TTL vs CMOS vs frequency

freq = np.logspace(0, 8, 300)  # 1 Hz to 100 MHz
VCC = 5.0

# 74LS (TTL): static power dominates
P_static_ttl = 2e-3  # 2 mW per gate
P_dynamic_ttl = 10e-12 * VCC**2 * freq  # small dynamic component
P_ttl = P_static_ttl + P_dynamic_ttl

# 74HC (CMOS at 5V)
P_static_cmos_5v = 2.5e-9 * VCC  # ~2.5 nA * 5V = 12.5 nW
C_PD = 22e-12  # 22 pF power dissipation capacitance
P_dynamic_cmos_5v = C_PD * VCC**2 * freq
P_cmos_5v = P_static_cmos_5v + P_dynamic_cmos_5v

# 74HC (CMOS at 3.3V)
VCC_33 = 3.3
P_static_cmos_33v = 2.5e-9 * VCC_33
P_dynamic_cmos_33v = C_PD * VCC_33**2 * freq
P_cmos_33v = P_static_cmos_33v + P_dynamic_cmos_33v

fig, ax = plt.subplots(figsize=(11, 6))

ax.loglog(freq, P_ttl * 1e3, 'r-', linewidth=2.5, label='74LS (TTL, 5V)')
ax.loglog(freq, P_cmos_5v * 1e3, 'b-', linewidth=2.5, label='74HC (CMOS, 5V)')
ax.loglog(freq, P_cmos_33v * 1e3, 'g-', linewidth=2.5, label='74HC (CMOS, 3.3V)')

# Mark key frequencies
for f_mark, label in [(1e3, '1 kHz'), (1e6, '1 MHz'), (1e8, '100 MHz')]:
    ax.axvline(x=f_mark, color='gray', linestyle=':', alpha=0.3)
    ax.text(f_mark, 20, label, fontsize=8, color='gray', ha='center')

ax.set_xlabel('Switching Frequency (Hz)', fontsize=12)
ax.set_ylabel('Power per Gate (mW)', fontsize=12)
ax.set_title('Power Consumption: TTL vs CMOS Logic Families', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, which='both')
ax.set_ylim(1e-6, 100)

# Annotate power savings
ax.annotate('At 1 kHz:\nCMOS uses\n~150,000x less power!',
            xy=(1e3, P_cmos_5v[np.argmin(np.abs(freq-1e3))] * 1e3),
            xytext=(1e4, 1e-4),
            fontsize=10, color='blue',
            arrowprops=dict(arrowstyle='->', color='blue', lw=1.5))

plt.tight_layout()
plt.show()

# Calculate savings at key frequencies
for f_val in [1e3, 1e6, 1e7]:
    idx = np.argmin(np.abs(freq - f_val))
    ratio = P_ttl[idx] / P_cmos_5v[idx]
    print(f'At {f_val:.0e} Hz: TTL uses {ratio:.0f}x more power than CMOS (5V)')

print()
# P = C * VCC^2 * f shows why lower VCC helps
ratio_v = (5.0/3.3)**2
print(f'Lowering VCC from 5V to 3.3V reduces dynamic power by {ratio_v:.1f}x')
print('This is because P_dynamic = C * VCC^2 * f -- power scales with voltage SQUARED.')

## Concept — Fan-Out, Propagation Delay, and Rise Time

### Fan-Out

**Fan-out** is how many inputs one output can drive reliably.

- **TTL fan-out** is limited by **current**: each input draws ~20-40 uA (HIGH) or ~0.4-1.6 mA (LOW). A TTL output can source/sink limited current, so fan-out ~ 10-20.
- **CMOS fan-out** is limited by **capacitance**: each input is a gate capacitor (~5 pF). More inputs = more capacitance = slower switching. Electrically, fan-out is nearly unlimited, but practically ~50 before speed degrades.

### Propagation Delay

**Propagation delay** (t_PD) is the time from an input change to the corresponding output change:
- **t_PLH**: delay for output going LOW-to-HIGH
- **t_PHL**: delay for output going HIGH-to-LOW

```
Input:  _____|^^^^^^^^|_____
              |        |
Output: ^^^^^ |   _____|^^^^^
              |  |
              |--| <-- t_PHL (this edge)
```

For 74HC at 5V: t_PD ~ 7-10 ns. Chaining 10 gates adds 70-100 ns of delay.

### Rise and Fall Time

How quickly the output transitions between LOW and HIGH:
- Determined by the output transistor driving the load capacitance
- Faster rise/fall = sharper edges = more electromagnetic interference (EMI)
- Slower rise/fall = less EMI but limits maximum operating frequency
- For 74HC: t_rise ~ 7 ns, t_fall ~ 7 ns

### Decoupling Capacitors

During switching, an IC draws a brief spike of current from VCC. This spike causes a voltage dip on the power rail. If the dip is large enough, other ICs on the same rail may malfunction.

**Solution**: place a 100 nF (0.1 uF) ceramic capacitor between VCC and GND, as close to each IC as possible. This capacitor acts as a local charge reservoir:

```
                   +------[IC VCC pin]
   VCC rail -------+
                   +--||--+
                   100nF  |
   GND rail -------+------+------[IC GND pin]
```

**Every IC gets its own decoupling cap. No exceptions.** This is the single most important practical rule in digital circuit design.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Visualize propagation delay and rise/fall time

t = np.linspace(0, 100, 2000)  # time in nanoseconds

# Input signal: step at t=10 ns, step back at t=60 ns
def step_signal(t, t_step, rise_time=1.0, vlow=0.0, vhigh=5.0):
    """Generate a signal with finite rise time."""
    return vlow + (vhigh - vlow) * 0.5 * (1 + np.tanh(4 * (t - t_step) / rise_time))

# Input with fast edges
Vin = step_signal(t, 10, rise_time=2) - step_signal(t, 60, rise_time=2) + step_signal(t, 60, rise_time=2, vhigh=0)
Vin = step_signal(t, 10, rise_time=2) * (1 - step_signal(t, 60, rise_time=2, vlow=0, vhigh=1))

# Output: inverted, delayed by tPD=8ns, with slower edges (rise_time=7ns)
tPD = 8  # propagation delay in ns
t_rise = 7  # rise/fall time
Vout_inv = 5.0 - (step_signal(t, 10 + tPD, rise_time=t_rise) * 
                   (1 - step_signal(t, 60 + tPD, rise_time=t_rise, vlow=0, vhigh=1)))

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 7), sharex=True)

# Input
ax1.plot(t, Vin, 'b-', linewidth=2.5)
ax1.set_ylabel('Input (V)', fontsize=12)
ax1.set_title('Propagation Delay and Signal Timing', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(-0.5, 5.5)
ax1.axhline(y=3.5, color='green', linestyle=':', alpha=0.4)
ax1.axhline(y=1.5, color='red', linestyle=':', alpha=0.4)
ax1.text(85, 3.7, 'V_IH', fontsize=9, color='green')
ax1.text(85, 1.0, 'V_IL', fontsize=9, color='red')

# Output (inverted)
ax2.plot(t, Vout_inv, 'r-', linewidth=2.5)
ax2.set_ylabel('Output (V)', fontsize=12)
ax2.set_xlabel('Time (ns)', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.set_ylim(-0.5, 5.5)

# Mark propagation delay
# t_PHL: input goes HIGH, output goes LOW
ax1.annotate('', xy=(10, 2.5), xytext=(10 + tPD, 2.5),
            arrowprops=dict(arrowstyle='<->', color='purple', lw=2))
ax2.annotate('', xy=(10, 2.5), xytext=(10 + tPD, 2.5),
            arrowprops=dict(arrowstyle='<->', color='purple', lw=2))
ax2.text(10 + tPD/2, 3.0, f't_PHL = {tPD} ns', ha='center', fontsize=10,
         color='purple', fontweight='bold')

# Mark rise time on output
# Find 10% and 90% points of the rising edge
ax2.annotate(f'Rise time\n~{t_rise} ns', xy=(68 + tPD, 2.5),
            xytext=(80, 2.5),
            fontsize=10, color='orange',
            arrowprops=dict(arrowstyle='->', color='orange', lw=1.5))

plt.tight_layout()
plt.show()

print('For 74HC00 at VCC = 5V (typical from datasheet):')
print(f'  Propagation delay: t_PHL = t_PLH = ~8 ns')
print(f'  Rise/fall time: ~7 ns')
print(f'  Maximum frequency: f_max = 1 / (2 * t_PD) ~ {1/(2*8e-9)/1e6:.0f} MHz (theoretical)')
print(f'  Practical max with margin: ~25 MHz')

## Experiment — Wiring Up the 74HC00 Quad NAND Gate

### Parts needed
- 1x 74HC00 (quad 2-input NAND gate, DIP-14 package)
- 1x 100 nF ceramic capacitor (decoupling)
- Bench power supply set to 5V
- Klein MM300 multimeter
- Breadboard + jumper wires

### 74HC00 Pinout (DIP-14)

```
        +---\/---+
   1A  -| 1   14 |- VCC
   1B  -| 2   13 |- 4B
   1Y  -| 3   12 |- 4A
   2A  -| 4   11 |- 4Y
   2B  -| 5   10 |- 3B
   2Y  -| 6    9 |- 3A
  GND  -| 7    8 |- 3Y
        +--------+

  Each gate: Y = NOT(A AND B) = NAND(A, B)
  Gate 1: pins 1, 2 -> 3
  Gate 2: pins 4, 5 -> 6
  Gate 3: pins 9, 10 -> 8
  Gate 4: pins 12, 13 -> 11
```

### Wiring Procedure

1. **Power first**: 
   - Pin 14 (VCC) to +5V rail
   - Pin 7 (GND) to ground rail
   - **100 nF capacitor** between pin 14 and pin 7, as close to the IC as possible

2. **Tie unused inputs**: Connect pins 4, 5, 9, 10, 12, 13 to either VCC or GND. **Never leave CMOS inputs floating!** (Floating inputs oscillate and waste power.)

3. **Test Gate 1**:
   - Pin 1 (1A) and Pin 2 (1B) are inputs
   - Pin 3 (1Y) is the output
   - Use jumper wires to connect inputs to VCC (+5V = HIGH) or GND (0V = LOW)
   - Measure output voltage at Pin 3 with multimeter

### Circuit on Breadboard

```
   +5V Rail ----+--------+---------+---- (to unused inputs)
                |        |
               [C]     [pin 14]
              100nF      |
                |    +---+---+
   GND Rail ----+----| 74HC00 |
                     +---+---+
                       [pin 7]
                         |
   GND Rail -------------+

   Test: connect pin 1 and pin 2 to VCC or GND
         measure pin 3 with multimeter
```

## Experiment — Verify the NAND Truth Table

### Procedure

For each input combination, connect the input pins and measure the output:

| Test | Pin 1 (A) | Pin 2 (B) | Pin 3 (Y) Expected | Pin 3 (Y) Measured |
|------|-----------|-----------|--------------------|-----------------|
| 1 | GND (0) | GND (0) | ~4.9V (HIGH) | _________ |
| 2 | GND (0) | +5V (1) | ~4.9V (HIGH) | _________ |
| 3 | +5V (1) | GND (0) | ~4.9V (HIGH) | _________ |
| 4 | +5V (1) | +5V (1) | ~0.0V (LOW) | _________ |

### What to measure and compare

1. **V_OH**: When output is HIGH, measure the exact voltage. The 74HC00 datasheet says V_OH >= 4.9V at VCC=5V. You should see something like 4.95-5.00V.

2. **V_OL**: When output is LOW (test 4), measure the exact voltage. Datasheet says V_OL <= 0.1V. You should see something like 0.01-0.05V.

3. **Supply current**: Measure the total supply current with multimeter in series with VCC. With all inputs static, you should see < 10 uA (the IC is barely consuming power).

### Bonus: What happens without the decoupling cap?

If you have the Fnirsi oscilloscope:
1. Connect oscilloscope probe to VCC rail (pin 14)
2. Toggle inputs rapidly (connect a wire to a signal generator at 1 MHz)
3. Observe VCC rail noise WITH the 100 nF cap
4. Remove the cap and observe again -- you should see spikes/ringing on VCC

This demonstrates why every IC needs its own decoupling capacitor.

## Experiment — Measuring Propagation Delay (Advanced)

This experiment requires the **Fnirsi 2C53T oscilloscope** and its built-in signal generator.

### Setup

Chain 3 of the 4 NAND gates in series (each wired as an inverter by tying inputs together):

```
                Gate 1           Gate 2           Gate 3
  Signal  --+--[NAND]--wire--+--[NAND]--wire--+--[NAND]--- Output
  Gen       |   1,2->3       |   4,5->6       |   9,10->8
  (1 MHz    |                |                |
   square)  |                |                |
            |                |                |
   CH1 -----+                                 +-------- CH2
```

### Procedure

1. Set the Fnirsi signal generator to output a **1 MHz square wave** (0-5V)
2. Connect CH1 to the input (signal generator output)
3. Connect CH2 to the output (pin 8, after 3 inverters)
4. Set oscilloscope timebase to 20 ns/div or 50 ns/div
5. Trigger on CH1 rising edge
6. Measure the time difference between CH1 edge and CH2 edge

### Expected Results

- 3 gates of delay at ~8 ns each = ~24 ns total propagation delay
- Output is inverted (3 inversions = net inversion)
- You should also see that the output edges are slightly slower than the input edges

**Note**: The Fnirsi 2C53T has a bandwidth of 50 MHz, which should be adequate to see 8 ns delays, though the edges may appear slightly rounded.

## Simulation — Build It in Falstad

### CMOS Inverter Chain

Build a chain of CMOS inverters to observe propagation delay.

**What to build:**
1. Open [Falstad (blank canvas)](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Build three CMOS inverters in series (each one is a PMOS + NMOS pair as described in the previous notebook)
3. Connect the output of each inverter to the input of the next
4. Drive the first input with a square wave (e.g. 1 MHz)
5. Right-click → **View in New Scope** at the input and at each inverter's output

**What to observe:**
- Watch the delay as the signal propagates through each gate — each stage adds a small delay
- Notice how the edges get slightly less sharp with each stage (parasitic capacitance at each node must charge/discharge)
- Toggle the clock speed to see how the chain behaves at different frequencies — at high enough frequency, the output can't keep up

### TTL vs CMOS: Build Both

**What to build:**
1. Open a [new blank canvas](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKCA)
2. Build a BJT-based inverter (simulating TTL): NPN transistor with a collector resistor to VCC, input to base through a resistor, output from collector
3. Build a CMOS inverter (PMOS + NMOS pair) next to it
4. Drive both with the same input

**What to observe:**
- Compare the output voltage swings: the CMOS inverter swings fully from 0V to VCC; the BJT inverter may not reach either rail as cleanly
- The BJT circuit draws continuous current through the collector resistor when the transistor is on — CMOS draws essentially zero static current
- This is why CMOS replaced TTL for most digital logic: lower power, better noise margins, rail-to-rail output

## Datasheet Connection

Every spec we discussed in this notebook comes directly from the datasheet. Here is where to find them on the **74HC00** datasheet:

### Section: Recommended Operating Conditions
```
Supply voltage (VCC):     2.0V to 6.0V
Input voltage:            0V to VCC
Operating temperature:    -40C to +85C (commercial)
```

### Section: DC Electrical Characteristics
```
At VCC = 5V, T = 25C:
  V_IH  >= 3.5V          Input HIGH threshold
  V_IL  <= 1.5V          Input LOW threshold  
  V_OH  >= 4.9V          Output HIGH (I_OH = -20 uA)
  V_OL  <= 0.1V          Output LOW (I_OL = 20 uA)
  I_CC  <= 4 uA          Quiescent supply current
```

### Section: AC Electrical Characteristics
```
At VCC = 5V, C_L = 50 pF, T = 25C:
  t_PHL = 8 ns (typ)     Propagation delay (HIGH to LOW)
  t_PLH = 8 ns (typ)     Propagation delay (LOW to HIGH)
  t_THL = 7 ns (typ)     Transition time (fall)
  t_TLH = 7 ns (typ)     Transition time (rise)
```

### What the numbers mean in practice

- **V_IH = 3.5V** at 5V: A 3.3V signal (HIGH = 3.2V) will NOT be recognized as HIGH. You need a level shifter or use 74HCT.
- **I_CC = 4 uA**: At rest, this IC draws 20 microwatts. You could run it on a coin cell battery for years.
- **t_PD = 8 ns**: At 10 MHz clock, one gate delay is 8% of the clock period. Chain 10 gates and you have consumed 80% of your timing budget.
- **C_L = 50 pF**: The test conditions assume 50 pF load. More capacitance = slower.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Propagation delay vs supply voltage (from 74HC00 datasheet values)

# Typical values from datasheet
VCC_data = np.array([2.0, 3.0, 4.5, 5.0, 6.0])  # V
tPD_data = np.array([36, 14, 9, 8, 7])  # ns (typical t_PHL)

# Interpolate for smooth curve
VCC_smooth = np.linspace(2.0, 6.0, 100)
tPD_smooth = np.interp(VCC_smooth, VCC_data, tPD_data)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Propagation delay vs VCC
ax1.plot(VCC_smooth, tPD_smooth, 'b-', linewidth=2)
ax1.plot(VCC_data, tPD_data, 'ro', markersize=8, label='Datasheet values')

for v, t in zip(VCC_data, tPD_data):
    ax1.annotate(f'{t} ns', xy=(v, t), xytext=(v + 0.1, t + 1.5),
                fontsize=9, color='red')

ax1.set_xlabel('Supply Voltage VCC (V)', fontsize=12)
ax1.set_ylabel('Propagation Delay (ns)', fontsize=12)
ax1.set_title('74HC00 Propagation Delay vs VCC', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 45)

ax1.annotate('Higher VCC = faster switching!\n(stronger gate drive)',
            xy=(4.5, 9), xytext=(2.5, 20),
            fontsize=10, color='blue',
            arrowprops=dict(arrowstyle='->', color='blue', lw=1.5))

# Maximum operating frequency vs VCC
fmax = 1 / (2 * tPD_smooth * 1e-9) / 1e6  # MHz
fmax_data = 1 / (2 * tPD_data * 1e-9) / 1e6

ax2.plot(VCC_smooth, fmax, 'g-', linewidth=2)
ax2.plot(VCC_data, fmax_data, 'ro', markersize=8, label='From datasheet tPD')

for v, f in zip(VCC_data, fmax_data):
    ax2.annotate(f'{f:.0f} MHz', xy=(v, f), xytext=(v + 0.1, f + 3),
                fontsize=9, color='red')

ax2.set_xlabel('Supply Voltage VCC (V)', fontsize=12)
ax2.set_ylabel('Max Frequency (MHz)', fontsize=12)
ax2.set_title('74HC00 Maximum Frequency vs VCC', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('Key insight: Running at lower VCC saves power (P ~ VCC^2) but the chip is SLOWER.')
print('This is the fundamental speed-power trade-off in CMOS design.')
print(f'  At 2.0V: tPD = {tPD_data[0]} ns, fmax ~ {fmax_data[0]:.0f} MHz')
print(f'  At 5.0V: tPD = {tPD_data[3]} ns, fmax ~ {fmax_data[3]:.0f} MHz')
print(f'  That is {tPD_data[0]/tPD_data[3]:.1f}x slower at 2V, but uses {(2.0/5.0)**2:.2f}x the power.')

## Checkpoint Questions

Test your understanding before moving on:

---

**Q1:** A 74HC output driving a 74HC input at VCC = 5V. The output is HIGH at 4.9V. What is the noise margin? How much noise voltage could corrupt the signal before the receiver misreads it?

<details>
<summary>Answer</summary>

The receiver's V_IH = 3.5V (at VCC=5V). Noise margin = V_OH - V_IH = 4.9V - 3.5V = 1.4V. The signal can tolerate up to 1.4V of noise (voltage drop) and still be recognized as HIGH. This is quite robust.

</details>

---

**Q2:** You have a 3.3V microcontroller (output HIGH = 3.3V) and a 74HC gate running at 5V (V_IH = 3.5V). Will the microcontroller's HIGH output be recognized as HIGH by the 74HC gate? What can you do about it?

<details>
<summary>Answer</summary>

No! The microcontroller's 3.3V output is below the 74HC's 3.5V input threshold. The gate may not recognize it as HIGH, or it may be in the undefined region, causing unreliable operation. Solutions: (1) Use a 74HCT gate instead, which has V_IH = 2.0V (TTL-compatible). (2) Use a level shifter IC. (3) Run the 74HC at 3.3V instead of 5V (then V_IH = 0.7 * 3.3V = 2.31V, which 3.3V exceeds).

</details>

---

**Q3:** Why does CMOS logic consume almost zero power at DC but significant power at high frequencies? Write the power equation.

<details>
<summary>Answer</summary>

P_total = P_static + P_dynamic. P_static is nearly zero because in CMOS, there is never a DC path from VCC to GND (one transistor in each complementary pair is always OFF). P_dynamic = C_PD x VCC^2 x f, where C_PD is the power dissipation capacitance, VCC is the supply voltage, and f is the switching frequency. Each switching event charges/discharges the load capacitance, drawing current from VCC. More switches per second = more power.

</details>

---

**Q4:** You are designing a circuit with ten 74HC00 gates. How many decoupling capacitors do you need, and where do they go?

<details>
<summary>Answer</summary>

The 74HC00 contains 4 gates per IC, so 10 gates requires 3 ICs (with 2 spare gates). You need at minimum 3 decoupling capacitors -- one for each IC package, placed as close as possible between each IC's VCC and GND pins. Use 100 nF ceramic capacitors. You may also want a bulk decoupling cap (10 uF electrolytic) near the power supply connector for the board.

</details>

---

**Q5:** The 74HC00 datasheet says propagation delay is 8 ns at 5V and 36 ns at 2V. Why does lower supply voltage make the chip slower?

<details>
<summary>Answer</summary>

The propagation delay is dominated by the time to charge and discharge the load capacitance through the MOSFET on-resistance. At lower VCC: (1) the gate-source voltage overdrive (VGS - Vth) is smaller, so the MOSFET has higher on-resistance and lower drive current; (2) the output still needs to swing the same fraction of VCC to cross the threshold. With less drive current and the same capacitance to charge, it takes longer. The speed roughly scales as (VCC - Vth).

</details>

---

**Q6:** What happens if you leave an unused CMOS input floating (not connected to anything)? Why is this a problem?

<details>
<summary>Answer</summary>

A floating CMOS input can pick up stray electromagnetic fields and oscillate between HIGH and LOW. Since the CMOS input is essentially a capacitor (extremely high impedance), even tiny induced currents can change the voltage. When the input is in the transition region (between V_IL and V_IH), both the NMOS and PMOS are partially on, creating a DC path from VCC to GND. This wastes significant power and generates heat -- potentially enough to damage the IC. Always tie unused CMOS inputs to VCC or GND.

</details>

---

## Summary

| Topic | Key Takeaway |
|-------|--------------|
| Logic family | Standardized voltage levels, speed, and power characteristics for interoperable ICs |
| TTL | BJT-based, 5V only, V_OH >= 2.4V, tight noise margins (0.4V) |
| CMOS (74HC) | MOSFET-based, 2-6V, rail-to-rail outputs, wide noise margins (~1.4V at 5V) |
| 74HCT | CMOS with TTL-compatible inputs -- bridges 3.3V/TTL to 5V CMOS |
| Level shifting | 3.3V CMOS cannot directly drive 5V CMOS (HC); use HCT or level shifter |
| Power | P = C_PD x VCC^2 x f -- CMOS power scales with frequency and voltage squared |
| Decoupling | Every IC needs a 100 nF cap between VCC and GND, close to the pins |
| Propagation delay | ~8 ns at 5V for 74HC; slower at lower VCC (speed-power trade-off) |

**Next up**: [02 -- Reading a Logic IC Datasheet](./02-reading-a-logic-ic-datasheet.ipynb) -- a complete walkthrough of the 74HC00 datasheet, section by section.